# Anisotropic Filtering Neural Network

In [1]:
import os
import random

import cv2 as cv

from anisotropic import anisodiff

### Create the filtered images that will be used to train the neural network as the expected output.

In [2]:
def convert_to_grayscale(image):
    return cv.cvtColor(image, cv.COLOR_BGR2GRAY)

In [3]:
def apply_anisotropic_diffusion(image, niter=50, kappa=20, gamma=0.2, step=(1., 1.), option=1, ploton=False):
    return anisodiff(image, niter=niter, kappa=kappa, gamma=gamma, step=step, option=option, ploton=ploton)

In [4]:
original_images_path = "data/original/"
filtered_images_path = "data/filtered/"

In [5]:
if not os.path.exists(filtered_images_path):
    os.makedirs(filtered_images_path)

In [6]:
original_images = [image for image in os.listdir(original_images_path)]

In [7]:
for image in original_images:
    image_path = os.path.join(original_images_path, image)
    image_filtered_path = os.path.join(filtered_images_path, image)
    
    if os.path.exists(image_filtered_path):
        continue
    
    image = cv.imread(image_path)
    gray_image = convert_to_grayscale(image)
    image_filtered = apply_anisotropic_diffusion(gray_image)
    
    cv.imwrite(image_filtered_path, image_filtered)

### Create windows for the neural network to learn from.

In [8]:
window_size = 64

In [9]:
def create_window_set():
    image_name = random.choice(original_images).split("/")[-1]
    
    image_x = cv.imread(os.path.join(original_images_path, image_name))
    image_y = cv.imread(os.path.join(filtered_images_path, image_name))
    
    width, height = image_x.shape[1], image_x.shape[0]
    
    x_start = random.randint(0, width - window_size)
    y_start = random.randint(0, height - window_size)
    x_end = x_start + window_size
    y_end = y_start + window_size
    
    image_x = image_x[y_start:y_end, x_start:x_end]
    image_y = image_y[y_start:y_end, x_start:x_end]
    
    return image_x, image_y   

In [ ]:
N = 500000

In [11]:
train_data = []

In [12]:
for i in range(N):
    image_x, image_y = create_window_set()
    
    train_data.append((image_x, image_y))